In [ ]:
# 시뮬레이션 프레임워크 설치
!pip install simpy
!pip freeze

import numpy as np
import pandas as pd
import datetime as dt

from numpy.random import triangular
from random import random, choice
from pytz import timezone

from dependencies import set_simulate


In [ ]:
# 주문 지역
TUNN_VALUE1 = 0.0066
TUNN_VALUE2 = 0.0022
MAX_SIZE = 1000

ORDER_REGION = {
    "방배1동": {
        "LAT": triangular(37.481877 - TUNN_VALUE1, 37.481877, 37.481877 + TUNN_VALUE1, MAX_SIZE),
        "LNG": triangular(127.005355 - TUNN_VALUE1, 127.005355 , 127.005355 + TUNN_VALUE1, MAX_SIZE),
    },
    "서초1동": {
        "LAT": triangular(37.489656 - TUNN_VALUE1, 37.489656, 37.489656 + TUNN_VALUE1, MAX_SIZE),
        "LNG": triangular(127.031045 - TUNN_VALUE1, 127.031045, 127.031045 + TUNN_VALUE1, MAX_SIZE),
    },
    "서초3동": {
        "LAT": triangular(37.487882 - TUNN_VALUE1, 37.487882, 37.487882 + TUNN_VALUE1, MAX_SIZE),
        "LNG": triangular(127.014304 - TUNN_VALUE1, 127.014304, 127.014304  + TUNN_VALUE1, MAX_SIZE),
    },
    "반포1동": {
        "LAT": triangular(37.508494 - TUNN_VALUE1, 37.508494, 37.508494 + TUNN_VALUE1, MAX_SIZE),
        "LNG": triangular(127.019902 - TUNN_VALUE1, 127.019902, 127.019902 + TUNN_VALUE1, MAX_SIZE),
    },
    "논현1동": {
        "LAT": triangular(37.508297 - TUNN_VALUE1, 37.508297, 37.508297 + TUNN_VALUE1, MAX_SIZE),
        "LNG": triangular(127.027481 - TUNN_VALUE1, 127.027481, 127.027481 + TUNN_VALUE1, MAX_SIZE),
    },
    "역삼1동": {
        "LAT": triangular(37.498435 - TUNN_VALUE1, 37.498435, 37.498435 + TUNN_VALUE1, MAX_SIZE),
        "LNG": triangular(127.034969 - TUNN_VALUE1, 127.034969 , 127.034969 + TUNN_VALUE1, MAX_SIZE),
    },
    "역삼2동": {
        "LAT": triangular(37.504282 - TUNN_VALUE1, 37.504282, 37.504282 + TUNN_VALUE1, MAX_SIZE),
        "LNG": triangular(127.031815 - TUNN_VALUE1, 127.031815, 127.031815 + TUNN_VALUE1, MAX_SIZE),
    },
    "신사동": {
        "LAT": triangular(37.521772 - TUNN_VALUE1, 37.521772, 37.521772 + TUNN_VALUE1, MAX_SIZE),
        "LNG": triangular(127.023581 - TUNN_VALUE1, 127.023581, 127.023581 + TUNN_VALUE1, MAX_SIZE),
    },
}

# 지점 정보
RESTAURANTS = [{
    "BRANCH_REGION": "seocho",
    "BRANCH_LAT": 37.498, 
    "BRANCH_LNG": 127.0248,
    "CHEF_COUNT": 15, 
    "COOKING_TIME": triangular(5, 7, 25, MAX_SIZE),
    # 피크에 시간당 80건
    "ORDER_INTERVAL_TIME": {
        'PEAK': triangular(0, 0.75, 1.5, MAX_SIZE),
        'PEAK_PRE': triangular(0, 3, 10, MAX_SIZE),
        'PEAK_OFF': triangular(0, 50, 70, MAX_SIZE)
    },
    "VALID_ORDER_REGION": ['서초1동', '반포1동','역삼1동','방배1동', '역삼2동'],
}, {
    "BRANCH_REGION": "sinsa",
    "BRANCH_LAT": 37.511036,
    "BRANCH_LNG": 127.020094,
    "CHEF_COUNT": 10,
    "COOKING_TIME": triangular(5, 8, 25, MAX_SIZE),
    # 피크에 시간당 60건
    "ORDER_INTERVAL_TIME": {
        'PEAK': triangular(0, 1, 1.5, MAX_SIZE), 
        'PEAK_PRE': triangular(0, 5, 10, MAX_SIZE),
        'PEAK_OFF': triangular(0, 60, 80, MAX_SIZE)
    },
    "VALID_ORDER_REGION": ['반포1동', '논현1동', '역삼1동', '역삼2동', '신사동', '서초3동'],
}]

# 피크타임 설정
STAND_TIME = 60
PEAK_TIME = {
    "LAUNCH": {
        "PRE": STAND_TIME * 1, # 11시
        "START": STAND_TIME * 1.5, # 11시 반
        "END": STAND_TIME * 2.5, # 13시 반
    },
    "DINNER": {
        "PRE": STAND_TIME * 7.5, # 18시 반
        "START": STAND_TIME * 8, # 19시
        "END": STAND_TIME * 10, # 21시
    },
}



In [ ]:
# 지점별 주문 처리 프로세스 등록
def set_real_order_df(RUN_TIME):
    
    real_df = pd.DataFrame(columns = ['orderDate', 'order_at', 'orderNo', 'room_no', 'site', 'deliveryType', 'origin_lat', 'origin_lng', 'lat', 'lng'])
    idx = 0
    
    for restaurant in RESTAURANTS:
        # 지점들은 각각의 현재 시간을 갖음
        now_time = 0

        while now_time < RUN_TIME:
            # 주문 데이터 생성
            
            # 주문 피크타임에 대한 추가시간 부여
            order_interval_time = 0
            is_launch_peck_pre = (now_time >= PEAK_TIME["LAUNCH"]["PRE"]) & (now_time <= PEAK_TIME["LAUNCH"]["START"])
            is_dinner_peck_pre = (now_time >= PEAK_TIME["DINNER"]["PRE"]) & (now_time <= PEAK_TIME["DINNER"]["START"])
            is_launch_peck = (now_time >= PEAK_TIME["LAUNCH"]["START"]) & (now_time <= PEAK_TIME["LAUNCH"]["END"])
            is_dinner_peck = (now_time >= PEAK_TIME["DINNER"]["START"]) & (now_time <= PEAK_TIME["DINNER"]["END"])

            if (is_launch_peck_pre | is_dinner_peck_pre):
                order_interval_time = restaurant["ORDER_INTERVAL_TIME"]["PEAK_PRE"]
            elif (is_launch_peck | is_dinner_peck):
                order_interval_time = restaurant["ORDER_INTERVAL_TIME"]["PEAK"]
            else:
                order_interval_time = restaurant["ORDER_INTERVAL_TIME"]["PEAK_OFF"]

            now_time += choice(order_interval_time)
            
            order_id = idx
            order_region = choice(restaurant['VALID_ORDER_REGION'])
            lat = choice(ORDER_REGION[order_region]['LAT'])
            lng = choice(ORDER_REGION[order_region]['LNG'])
            
            real_df.loc[idx, 'order_at'] = now_time
            real_df.loc[idx, 'orderNo'] = idx
            real_df.loc[idx, 'orderDate'] = dt.datetime(2022,3,1,10,0,0,0) + dt.timedelta(minutes = now_time)
            real_df.loc[idx, 'site'] = restaurant["BRANCH_REGION"]
            real_df.loc[idx, 'room_no'] = "{}_{:02d}".format(restaurant["BRANCH_REGION"], np.random.randint(0, restaurant["CHEF_COUNT"], 1)[0])
            real_df.loc[idx, 'deliveryType'] = 'delivery'
            
            real_df.loc[idx, 'origin_lat'] = restaurant['BRANCH_LAT']
            real_df.loc[idx, 'origin_lng'] = restaurant['BRANCH_LNG']
            
            order_region = choice(restaurant['VALID_ORDER_REGION'])
            real_df.loc[idx, 'lat'] = choice(ORDER_REGION[order_region]['LAT'])
            real_df.loc[idx, 'lng'] = choice(ORDER_REGION[order_region]['LNG'])
            
            idx += 1
                
    return real_df

class bundleSimGAoperator():
    def __init__(self, max_rider, *args, **kwargs):
        self.max_rider = max_rider
        super(bundleSimGAoperator, self).__init__(*args, **kwargs)
    
    def execute(self):
        run_time = 600.0

        run_at = dt.datetime.now()
        start_at = dt.datetime(2022,3,1,10,0,0,0)
#         end_at = start_at + dt.timedelta(minutes = run_time)

        real_df = set_real_order_df(run_time)

        simulate_delivery = set_simulate.simDelivery(run_at, max_rider = self.max_rider, run_time = run_time)
        simulate_delivery.set_order(start_at, real_df)


In [ ]:

RUN_TIME = 600

real_df = set_real_order_df(600)

print(real_df.groupby(['site']).agg({'order_at' : 'count', 'room_no' : lambda x : x.nunique()}))
print(real_df[real_df['site'] == 'seocho'].head(50))
print(real_df[real_df['site'] == 'sinsa'].head(50))

MAX_RIDER = 20

sim_ga = bundleSimGAoperator(max_rider = 20)
sim_ga.execute()

